In [18]:
import psycopg2 as pg2
from dotenv import load_dotenv
import os
import pandas as pd
import sqlalchemy

import requests
from datetime import date
import time

In [19]:
def get_magic_transfers_latest_block():

    conn = pg2.connect(
        host= "database-2.c2bw3zzer4hr.us-west-1.rds.amazonaws.com",
        port = 5432,
        user = 'postgres',
        password = AWS_DB_PASSWORD,
        database = "magic_treasure_db"
    )

    cur = conn.cursor()

    ##Get the latest blocknumber so we can query alchemy more efficiently
    sql_query = f"""
    SELECT 
        max(block_number) as latest_retrieved_block
        
    FROM magic_transfers mt
    """
    cur.execute(sql_query)

    data = cur.fetchone()

    if data[0] is None:
        return 0
    else:
        return data[0]
    
def get_response_wo_pagekey(start_block, ALCHEMY_API_KEY_TREASURE):
  url = f"https://arb-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY_TREASURE}"
  payload = {
      "id": 1,
      "jsonrpc": "2.0",
      "method": "alchemy_getAssetTransfers",
      "params": [
          {
              "fromBlock": f"{start_block}",
              "toBlock": "latest",
              "contractAddresses": ["0x539bde0d7dbd336b79148aa742883198bbf60342"],
              "category": ["erc20"],
              "withMetadata": True,
              "excludeZeroValue": False,
              "maxCount": "0x3e8",
              "order": "asc"
          }
      ]
  }
  headers = {
      "Accept": "application/json",
      "Content-Type": "application/json"
  }

  response = requests.post(url, json=payload, headers=headers)
  if 'error' in response.json():
    print(response.json()['error'])
  else:
    try:
        response=response.json()['result']
    except:
        response=response.json()
        print(response.keys())
  # print(response.text)
    return response

def get_response_w_pgkey(ALCHEMY_API_KEY_TREASURE, start_block, _page_key):
    url = f"https://arb-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY_TREASURE}"
  # contract_address = "0x539bde0d7dbd336b79148aa742883198bbf60342"
    payload = {
      "id": 1,
      "jsonrpc": "2.0",
      "method": "alchemy_getAssetTransfers",
      "params": [
          {
              "fromBlock": f"{start_block}",
              "toBlock": "latest",
              "contractAddresses": ["0x539bde0d7dbd336b79148aa742883198bbf60342"],
              "category": ["erc20"],
              "withMetadata": True,
              "excludeZeroValue": False,
              "maxCount": "0x3e8",
              "order": "asc",
              "pageKey": f"{_page_key}"
          }
      ]
  }
    headers = {
      "Accept": "application/json",
      "Content-Type": "application/json"
  }
    response = requests.post(url, json=payload, headers=headers)
    try:
        response = response.json()['result']
        # print(response)
        return response
    except:
        print(f"didn't work!\n{response}")
        return response.json()


In [21]:
# print (get_magic_transfers_latest_block())

load_dotenv()
start_time = time.time()
####------------------------------------------------------------------------------------####
####                             Connect to DB                                          ####
####------------------------------------------------------------------------------------####
AWS_DB_PASSWORD = os.getenv("AWS_DB_PASSWORD")
ALCHEMY_API_KEY_TREASURE = os.getenv("ALCHEMY_API_KEY_TREASURE")

latest_retrieved_block = int(get_magic_transfers_latest_block())

# print(latest_retrieved_block)
# latest_retrieved_block = 29503518
latest_block_hex = hex(latest_retrieved_block)

print("Contract: 0x539bde0d7dbd336b79148aa742883198bbf60342")
page_key = None
concat_num=0
skip_other_pages=False
df_transfers = pd.DataFrame()

response_0 = get_response_wo_pagekey(latest_block_hex,  ALCHEMY_API_KEY_TREASURE)
print(response_0.keys())
if 'pageKey' in response_0:
    page_key = response_0['pageKey']
else:
    print('Not enough for second page')
    skip_other_pages = True
df_transfers = pd.concat([df_transfers, pd.json_normalize(response_0['transfers'])], axis=0)

while True:
    if not skip_other_pages:
        response=get_response_w_pgkey(ALCHEMY_API_KEY_TREASURE, latest_block_hex, page_key)
        # print(f"\n******************\nLength: {len(response['transfers'])}")
        if 'transfers' in response:
            
            df_new = pd.json_normalize(response['transfers'])
            df_transfers = pd.concat([df_transfers, df_new], axis=0)
        else:
            break
        try:
            page_key=response['pageKey']
        except:
            break

        concat_num = concat_num + 1
        if concat_num % 100 == 0:
            print(f"Iteration Number: {concat_num}")
        # print(page_key)
    else:
        break
df_transfers=df_transfers.drop(['uniqueId', 'tokenId', 'erc1155Metadata', 'erc721TokenId', 'rawContract.decimal', 'rawContract.value', 'category', 'asset'], axis=1)

Contract: 0x539bde0d7dbd336b79148aa742883198bbf60342
dict_keys(['transfers', 'pageKey'])
Iteration Number: 100
Iteration Number: 200
Iteration Number: 300
Iteration Number: 400
Iteration Number: 500
Iteration Number: 600
Iteration Number: 700
Iteration Number: 800
Iteration Number: 900
Iteration Number: 1000
Iteration Number: 1100
Iteration Number: 1200
Iteration Number: 1300
Iteration Number: 1400
Iteration Number: 1500
Iteration Number: 1600
Iteration Number: 1700
Iteration Number: 1800
Iteration Number: 1900
Iteration Number: 2000
Iteration Number: 2100
Iteration Number: 2200
Iteration Number: 2300
Iteration Number: 2400
Iteration Number: 2500
Iteration Number: 2600
Iteration Number: 2700
Iteration Number: 2800
Iteration Number: 2900
Iteration Number: 3000
Iteration Number: 3100
Iteration Number: 3200


In [22]:
df_transfers

,blockNum,hash,from,to,value,rawContract.address,metadata.blockTimestamp
0,0x1ef22d,0x3d7c88327ec852ebf30b59b8ae4e42865e8fc71c057f...,0x0000000000000000000000000000000000000000,0xa4b1838cb086dddafa655f247716b502e87a0672,4.200000,0x539bde0d7dbd336b79148aa742883198bbf60342,2021-10-08T19:02:23.000Z
1,0x1fb543,0x67384ced75282bc4cee72708f6e1ec5c577376407fa4...,0x0000000000000000000000000000000000000000,0xa82fcefd303fa68864b787a5f118b09ce5a4c93d,50.000000,0x539bde0d7dbd336b79148aa742883198bbf60342,2021-10-10T10:56:05.000Z
2,0x1fb99a,0x81b729a1634fb59dcc1a769963f41dc227e1826eeea2...,0x0000000000000000000000000000000000000000,0xe8d46499994f4cf0f80feea5cfc9d9d502985286,2177.000000,0x539bde0d7dbd336b79148aa742883198bbf60342,2021-10-10T11:35:00.000Z
3,0x204179,0xec1b7b175a2d3a557a234c3a4e744a30010bea996cd1...,0x0000000000000000000000000000000000000000,0x8a83716acd66d9e1fb18c9b79540b72e04f80ac0,1551.848954,0x539bde0d7dbd336b79148aa742883198bbf60342,2021-10-11T11:03:55.000Z
4,0x207080,0x0ae48d8d59cf9cf85c495fc0d53b79cd347a38411a7d...,0x0000000000000000000000000000000000000000,0x9dcc35ae915926f7f5e8c624254d91f755d55b71,193256.857645,0x539bde0d7dbd336b79148aa742883198bbf60342,2021-10-11T18:20:48.000Z
...,...,...,...,...,...,...,...
965,0x1c23f1a,0x381f011059fe329f0079cb1afd91cf3b1cf4deb3a94f...,0x978a2e2d98ca029ab0f7268cb069528333d748c1,0xb7e50106a5bd3cf21af210a755f9c8740890a8c9,3.852264,0x539bde0d7dbd336b79148aa742883198bbf60342,2022-10-11T05:51:53.000Z
966,0x1c23f73,0x6ea65ddeaea4382f81e55d0b06310be193fbcbe43bae...,0xe37523f553606c6bbb0d5bd78da6c760b368ca2f,0xb7e50106a5bd3cf21af210a755f9c8740890a8c9,230.594564,0x539bde0d7dbd336b79148aa742883198bbf60342,2022-10-11T05:52:51.000Z
967,0x1c23feb,0xe9937b6f765d6c61bc0153635af908a9369389a050b1...,0xb7e50106a5bd3cf21af210a755f9c8740890a8c9,0xdd94018f54e565dbfc939f7c44a16e163faab331,51.555901,0x539bde0d7dbd336b79148aa742883198bbf60342,2022-10-11T05:54:17.000Z
968,0x1c23feb,0xe9937b6f765d6c61bc0153635af908a9369389a050b1...,0xdd94018f54e565dbfc939f7c44a16e163faab331,0x94796f85b5390f9ffcae4813ea179e083b3dfad0,51.555901,0x539bde0d7dbd336b79148aa742883198bbf60342,2022-10-11T05:54:17.000Z


In [23]:
def to_hex(x):
    return int(x, base=16)
df_transfers_numeric = df_transfers.copy()
df_transfers_numeric.loc[:, 'blockNum'] = df_transfers_numeric['blockNum'].apply(to_hex)

df_transfers_numeric.rename(columns={'blockNum':'block_number', 'rawContract.address':'contract_address', 'metadata.blockTimestamp':'block_timestamp', 'from':'from_address', 'to':'to_address'}, inplace=True)
df_transfers_numeric['block_timestamp'] = pd.to_datetime(df_transfers_numeric['block_timestamp']).dt.tz_convert(None)
# df_transfers_numeric.datetime.dt.tz_convert(None)
df_transfers_numeric

,block_number,hash,from_address,to_address,value,contract_address,block_timestamp
0,2028077,0x3d7c88327ec852ebf30b59b8ae4e42865e8fc71c057f...,0x0000000000000000000000000000000000000000,0xa4b1838cb086dddafa655f247716b502e87a0672,4.200000,0x539bde0d7dbd336b79148aa742883198bbf60342,2021-10-08 19:02:23
1,2078019,0x67384ced75282bc4cee72708f6e1ec5c577376407fa4...,0x0000000000000000000000000000000000000000,0xa82fcefd303fa68864b787a5f118b09ce5a4c93d,50.000000,0x539bde0d7dbd336b79148aa742883198bbf60342,2021-10-10 10:56:05
2,2079130,0x81b729a1634fb59dcc1a769963f41dc227e1826eeea2...,0x0000000000000000000000000000000000000000,0xe8d46499994f4cf0f80feea5cfc9d9d502985286,2177.000000,0x539bde0d7dbd336b79148aa742883198bbf60342,2021-10-10 11:35:00
3,2113913,0xec1b7b175a2d3a557a234c3a4e744a30010bea996cd1...,0x0000000000000000000000000000000000000000,0x8a83716acd66d9e1fb18c9b79540b72e04f80ac0,1551.848954,0x539bde0d7dbd336b79148aa742883198bbf60342,2021-10-11 11:03:55
4,2125952,0x0ae48d8d59cf9cf85c495fc0d53b79cd347a38411a7d...,0x0000000000000000000000000000000000000000,0x9dcc35ae915926f7f5e8c624254d91f755d55b71,193256.857645,0x539bde0d7dbd336b79148aa742883198bbf60342,2021-10-11 18:20:48
...,...,...,...,...,...,...,...
965,29507354,0x381f011059fe329f0079cb1afd91cf3b1cf4deb3a94f...,0x978a2e2d98ca029ab0f7268cb069528333d748c1,0xb7e50106a5bd3cf21af210a755f9c8740890a8c9,3.852264,0x539bde0d7dbd336b79148aa742883198bbf60342,2022-10-11 05:51:53
966,29507443,0x6ea65ddeaea4382f81e55d0b06310be193fbcbe43bae...,0xe37523f553606c6bbb0d5bd78da6c760b368ca2f,0xb7e50106a5bd3cf21af210a755f9c8740890a8c9,230.594564,0x539bde0d7dbd336b79148aa742883198bbf60342,2022-10-11 05:52:51
967,29507563,0xe9937b6f765d6c61bc0153635af908a9369389a050b1...,0xb7e50106a5bd3cf21af210a755f9c8740890a8c9,0xdd94018f54e565dbfc939f7c44a16e163faab331,51.555901,0x539bde0d7dbd336b79148aa742883198bbf60342,2022-10-11 05:54:17
968,29507563,0xe9937b6f765d6c61bc0153635af908a9369389a050b1...,0xdd94018f54e565dbfc939f7c44a16e163faab331,0x94796f85b5390f9ffcae4813ea179e083b3dfad0,51.555901,0x539bde0d7dbd336b79148aa742883198bbf60342,2022-10-11 05:54:17


In [24]:
df_transfers_numeric.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3210970 entries, 0 to 969
Data columns (total 7 columns):
 #   Column            Dtype         
---  ------            -----         
 0   block_number      int64         
 1   hash              object        
 2   from_address      object        
 3   to_address        object        
 4   value             float64       
 5   contract_address  object        
 6   block_timestamp   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 196.0+ MB


In [25]:
#Define connection variables
host= "database-2.c2bw3zzer4hr.us-west-1.rds.amazonaws.com"
port = 5432
user = 'postgres'
password = AWS_DB_PASSWORD
database = "magic_treasure_db"

postgres_str = f'postgresql://{user}:{password}@{host}:{port}/{database}'

engine = sqlalchemy.create_engine(postgres_str)

In [26]:
df_transfers_numeric.to_sql('magic_transfers', con=engine, index=False, if_exists='append', chunksize=10000)

321970